In [43]:
# Connect to Azure Machine Learning Workspace 
import json
from azureml.core import Workspace

ws = Workspace.from_config()

# make sure we have a scripts dir for the code to run our jobs.
import os
scripts_dir = "./scripts"
os.makedirs(scripts_dir, exist_ok=True)

config_file = ".azureml/config.json"
config = json.load(open(config_file, 'r'))

for required_key in ['subscription_id', 'resource_group', 'workspace_name', 'storage_account_key', 'storage_account_name']:
    if not required_key in config:
        print(f"### Error: please add a {required_key} to {config_file}")

storage_account_key = config['storage_account_key']    
storage_account_name = config['storage_account_name']

In [44]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [45]:
# Get a handle to the workspace
ml_client = MLClient.from_config(
    credential=credential,
    path='./.azureml/config.json'
)

Found the config file in: .azureml\config.json


In [46]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

face_data = Data(
    name="face_segmentation_dataset",
    path="https://nasfacemodels.blob.core.windows.net/downloads/099000.zip",
    type=AssetTypes.URI_FILE,
    description="Dataset for quantizing the model",
    tags={"source_type": "web", "source": "azure"},
    version="1.0.1",
)

face_data = ml_client.data.create_or_update(face_data)
print(
    f"Dataset with name {face_data.name} was registered to workspace, the dataset version is {face_data.version}"
)

Dataset with name face_segmentation_dataset was registered to workspace, the dataset version is 1.0.1


In [47]:
from azure.ai.ml.entities import AzureBlobDatastore
from azure.ai.ml.entities._credentials import AccountKeyConfiguration

# Register the blob store container for storing our models
data_store = AzureBlobDatastore(
    name="nassnpe205test",
    description="Datastore pointing to our dataset container.",
    account_name=storage_account_name,
    container_name="models",
    credentials=AccountKeyConfiguration(
        account_key=storage_account_key
    ),
)

ml_client.create_or_update(data_store)

AzureBlobDatastore({'type': <DatastoreType.AZURE_BLOB: 'AzureBlob'>, 'name': 'nassnpe205test', 'description': 'Datastore pointing to our dataset container.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/c8b7f913-60fb-4759-a310-fc5630e56f99/resourceGroups/snpe-aml-rg/providers/Microsoft.MachineLearningServices/workspaces/snpe-aml-workspace/datastores/nassnpe205test', 'Resource__source_path': None, 'base_path': 'd:\\git\\microsoft\\image_segmentation\\snpe\\notebooks\\quantize', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x0000017746404D30>, 'credentials': {'type': 'account_key'}, 'container_name': 'models', 'account_name': 'nassnpe205test', 'endpoint': 'core.windows.net', 'protocol': 'https'})

In [48]:
# This is the path shown in the Data asset we created to point to this model...
subscription = ml_client.subscription_id
rg = ml_client.resource_group_name
ws_name = ml_client.workspace_name

#datastore_path = f"azureml://datastores/nasfacemodels/paths/Deci1"
datastore_path = f"azureml://datastores/nassnpe205test/paths/Deci2"
 
model_path = f"{datastore_path}/deci_optimized_2.onnx"
dlc_path = f"{datastore_path}/model.dlc"
quant_dlc_path = f"{datastore_path}/model.quant.dlc"
dlc_info_path = f"{datastore_path}/model.info.txt"
quant_dlc_info_path = f"{datastore_path}/model.quant.info.txt"

In [49]:
from azure.ai.ml.entities import AmlCompute

snpe_cluster = "snpe-compute"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(snpe_cluster)
    print(
        f"Found your Kubernetes cluster named {snpe_cluster}, awesome!"
    )

except Exception:
    print(f"Computer cluster named {snpe_cluster} was not found ...")

Found your Kubernetes cluster named snpe-compute, awesome!


In [50]:
from azure.ai.ml.entities import Environment

custom_env_name = "quantization"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for running the quantizer pipeline",    
    image="snpecontainerregistry001.azurecr.io/snpe-2.5.0",
    version="1.3",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name quantization is registered to workspace, the environment version is 1.3


In [51]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

data_prep_component = command(
    name="data_prep",
    display_name="Data preparation for quantization",
    description="Unzips the dataset and converts to .bin format",
    inputs={
        "data": Input(type="uri_folder")
    },
    outputs= {
        "quant_data": Output(type="uri_folder", mode="rw_mount")
    },

    # The source folder of the component
    code=scripts_dir,
    command="""python3 data_prep.py \
            --data ${{inputs.data}} \
            --output ${{outputs.quant_data}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [52]:
convert_component = command(
    name="convert",
    display_name="Convert .onnx to .dlc",
    description="Converts the onnx model to dlc format",
    inputs={
        "model": Input(type="uri_file")
    },
    outputs= {
        "dlc": Output(type="uri_file", path=dlc_path, mode="rw_mount")
    },

    # The source folder of the component
    code=scripts_dir,
    command="""python3 convert.py \
            --model ${{inputs.model}} \
            --output ${{outputs.dlc}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [53]:
quant_component = command(
    name="quantize_model",
    display_name="Quantize the Model",
    description="Runs snpe-dlc-quant on the model using the prepared quantization dataset",
    inputs={
        "data": Input(type="uri_folder"),
        "list_file": Input(type="string"),
        "model": Input(type="uri_folder")
    },
    outputs= {
        "quant_model": Output(type="uri_file", path=quant_dlc_path, mode="rw_mount")
    },

    # The source folder of the component
    code=scripts_dir,
    command="""python3 quantize.py \
            --data ${{inputs.data}} \
            --list_file ${{inputs.list_file}} \
            --model ${{inputs.model}} \
            --output ${{outputs.quant_model}} \
            """,
    environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
)

In [54]:
def info_component(output_path):
    return command(
        name="model_info",
        display_name="Get model metrics",
        description="Runs snpe-dlc-info on the input .dlc model",
        inputs={
            "model": Input(type="uri_folder")
        },
        outputs= {
            "info": Output(type="uri_file", path=output_path, mode="rw_mount")
        },

        # The source folder of the component
        code=scripts_dir,
        command="""python3 dlc_info.py \
                --model ${{inputs.model}} \
                --output ${{outputs.info}} \
                """,
        environment=f"{pipeline_job_env.name}:{pipeline_job_env.version}",
    )

In [55]:
# the dsl decorator tells the sdk that we are defining an Azure ML pipeline
from azure.ai.ml import dsl, Input, Output

@dsl.pipeline(
    compute=snpe_cluster,
    description="Quantization pipeline",
)
def quantization_pipeline(
    pipeline_job_data_input,
    model_input
):
    # using data_prep_function like a python call with its own inputs
    data_prep_job = data_prep_component(
        data=pipeline_job_data_input
    )

    # convert onnx to dlc
    convert_job = convert_component(
        model=model_input
    )

    # get the dlc info on the converted model
    info_job = info_component(dlc_info_path)(
        model=convert_job.outputs.dlc
    )

    # quantize the dlc model
    quant_job = quant_component(
        data=data_prep_job.outputs.quant_data,
        list_file='input_list.txt',
        model=convert_job.outputs.dlc
    )

    # get the dlc info on quantized model
    info_job = info_component(quant_dlc_info_path)(
        model=quant_job.outputs.quant_model
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "pipeline_job_model": convert_job.outputs.dlc,
        "pipeline_job_quant_model": quant_job.outputs.quant_model,
        "pipeline_job_info": info_job.outputs.info
    }

In [60]:
# Let's instantiate the pipeline with the parameters of our choice
pipeline = quantization_pipeline(
    pipeline_job_data_input=Input(type="uri_file", path=face_data.path),
    model_input=Input(type="uri_file", path=model_path)
)

# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="quantization_test_run",
)

In [57]:
import webbrowser
# open the pipeline in web browser
webbrowser.open(pipeline_job.services["Studio"].endpoint)

True

In [58]:
ml_client.jobs.download(pipeline_job.name, output_name='pipeline_job_info')

In [59]:
print(pipeline_job)

name: strong_pasta_wgv176q8tw
display_name: quantization_pipeline
description: Quantization pipeline
type: pipeline
inputs:
  pipeline_job_data_input:
    mode: ro_mount
    type: uri_file
    path: azureml:https://nasfacemodels.blob.core.windows.net/downloads/099000.zip
  model_input:
    mode: ro_mount
    type: uri_file
    path: azureml://datastores/nassnpe205test/paths/Deci2/deci_optimized_2.onnx
outputs:
  pipeline_job_model:
    mode: rw_mount
    type: uri_file
    path: azureml://datastores/nassnpe205test/paths/Deci2/model.dlc
  pipeline_job_quant_model:
    mode: rw_mount
    type: uri_file
    path: azureml://datastores/nassnpe205test/paths/Deci2/model.quant.dlc
  pipeline_job_info:
    mode: rw_mount
    type: uri_file
    path: azureml://datastores/nassnpe205test/paths/Deci2/model.quant.info.txt
jobs:
  data_prep_job:
    type: command
    inputs:
      data:
        path: ${{parent.inputs.pipeline_job_data_input}}
    outputs:
      quant_data:
        mode: rw_mount
    